## Transverse Ising model in 2D - The FK #2 cost approach

The basic components of this rudimentary example:
- Variational approximation of the rates $\Gamma$ with a CNN, $\Gamma^{(v)} = \operatorname{CNN}_v(s)$ 
- Generator of trajectories with fixed endpoints
- Optimisation routine (gradient descent for now)
- Loss implementation
- All of the architecture necessary by the loss implementation


In [4]:
# Jax imports
import jax
import jax.numpy as jnp
from jax.experimental import stax
from jax.experimental.stax import (BatchNorm, Conv, Dense, Flatten, Relu, LogSoftmax)

import numpy as np
import matplotlib.pyplot as plt

In [5]:
""" Parameters of the example """

# Physical parameters
dim = 6 # size of lattice of the Ising model
J = 1
g = 1
T = 10 # full time

# Model hyperparameters
batch_size = 28

# Optimisation parameters
alpha = 1e-3



### Implementation of the log RN loss.
$$
\log \left(\frac{d P_{v}}{d P_{F K}}(k(t))\right)=\int V(k(t))+\sum_{l \neq k(t)}\left(\Gamma_{k(t) \rightarrow l}-\Gamma_{k(t) \rightarrow l}^{(v)}\right) d t+\sum_{n} \log \left(\frac{\Gamma_{k^{(n)} \rightarrow k^{(n+1)}}^{(v)}}{\Gamma_{k^{(n)} \rightarrow k^{(n+1)}}}\right)-E_{0} T-\log \left(\frac{\varphi_{k^{(N)}}}{\varphi_{k^{(0)}}}\right)
$$

In [6]:
def loss(params, trajectory, J, g):
    
    logRN = 0.0
    
    # 1st term - Integral of V(k(t))
    
    # 2nd term
    
    # 3rd term
    
    return None
    

### Implementation of the Ising model helper functions
The transverse field Ising model is the simplest model we will look at, the Hamiltonian
$$
H_{\mathrm{Ising}}=-J \sum_{\langle i, j\rangle} \sigma_z^{i} \sigma_z^{j}-g \sum_{i} \sigma_x^{i},
$$
is stoquastic if $g > 0$ and has a very simple passive transition rate matrix
$$
\Gamma_{s \rightarrow s^{\prime}}=g.
$$
The potential is also rather simple
$$
\begin{aligned} 
V(s) &=-\sum_{s^{\prime} \neq s} \Gamma_{s \rightarrow s^{\prime}}+H_{s s} \\ &=-g N-J \sum_{\langle i, j\rangle} s_{i} s_{j} 
\end{aligned}
$$

In [7]:
def V_ising(J):
    """
    Returns the Ising potential for square lattice with periodic boundary conditions
    """
    def pot(s, passive_rates):
        z = s
        zu = jnp.roll(z, 1, axis=-2)
        zr = jnp.roll(x, -1, axis=-1)
        
        # first term
        potential = (2*z - 1) * (2*zu - 1)
        potential += (2*z - 1) * (2*zr - 1) 
        potential = -J*potential.sum(axis=(-1, -2))

        #second term
        potential -= passive_rates.sum(axis=(-1, -2, -3))
        
        return potential

    return pot

def passive_ising(g):
    """
    Returns the passive rates for square Ising model with p.b.c's
    """
    def passive_rates(s):
        z = s.unsqueeze(dim=-3)
        rates = g*jnp.ones_like(z)
        return rates
    
    return passive_rates


### Continuous time Markov Chain
Using the holding time definition, where the transition probability is 
$$
P_{s\rightarrow s^\prime} = \frac{\Gamma_{s\rightarrow s^\prime}}{\sum_{s \neq s^\prime}\Gamma_{s\rightarrow s^\prime}}
$$
and the transition time is distributed with exponential probability
$$
\tau \sim \operatorname{Exp}\left(\sum_{x^{\prime}} \Gamma_{x \rightarrow x^{\prime}}\right)
$$

### Variational approximation to the transition rates
Just implement the a CNN variant that respects p.b.c's. See:
1) T. S. Cohen and M. Welling, “Group Equivariant Convolutional Networks”, arXiv:1602.07576 (2016).

2) S. Dieleman, J. De Fauw, and K. Kavukcuoglu, “Exploiting Cyclic Symmetry in Convolutional Neural Networks”, arXiv:1602.02660 (2016).